# Shifting and Scaling

In [1]:
import numpy as np
import astropy
import ccdproc
from ccdproc import CCDData, Combiner
from astropy import units as u
from astropy.visualization import SqrtStretch
import matplotlib.pyplot as plt 
from matplotlib.colors import LogNorm
from photutils import centroid_com, centroid_1dg, centroid_2dg
from photutils import CircularAperture
from photutils import aperture_photometry
from photutils import Background2D
from photutils import MedianBackground
from scipy.ndimage import shift
import pandas as pd
import gc 
gc.enable()

C:\Users\Jedd\AppData\Local\Temp\ipykernel_5924\2038989987.py:9: DeprecationWarning: `photutils.centroid_com` is a deprecated alias for `photutils.centroids.centroid_com`. Instead, please use `from photutils.centroids import centroid_com` to silence this warning.
  from photutils import centroid_com, centroid_1dg, centroid_2dg
C:\Users\Jedd\AppData\Local\Temp\ipykernel_5924\2038989987.py:9: DeprecationWarning: `photutils.centroid_1dg` is a deprecated alias for `photutils.centroids.centroid_1dg`. Instead, please use `from photutils.centroids import centroid_1dg` to silence this warning.
  from photutils import centroid_com, centroid_1dg, centroid_2dg
C:\Users\Jedd\AppData\Local\Temp\ipykernel_5924\2038989987.py:9: DeprecationWarning: `photutils.centroid_2dg` is a deprecated alias for `photutils.centroids.centroid_2dg`. Instead, please use `from photutils.centroids import centroid_2dg` to silence this warning.
  from photutils import centroid_com, centroid_1dg, centroid_2dg


In [2]:
images = ccdproc.ImageFileCollection(".") #loading in image files
#print(dir(images))
#print(images.files) #Printing the loaded files

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_all_keywords', '_dict_from_fits_header', '_ext', '_filenames', '_files', '_find_fits_by_reading', '_find_keywords_by_values', '_fits_files_in_directory', '_fits_summary', '_generator', '_get_files', '_glob_exclude', '_glob_include', '_location', '_paths', '_set_column_name_case_to_match_keywords', '_summary', 'ccds', 'data', 'ext', 'files', 'files_filtered', 'filter', 'glob_exclude', 'glob_include', 'hdus', 'headers', 'keywords', 'location', 'refresh', 'sort', 'summary', 'values']
['NGC_2323_B_average.fits', 'NGC_2323_B_median.fits', 'NGC_2323_R_average.fits', 'NGC_2323_R_median.fits', 'NGC_2323_V_average.fits', 'NGC_2323_V_median.fits', 'proc_N

Before loading in our images to the notebook, we manually checked our images using DS9 and removed any images that were blurred or had trailed stars as this would effect the combine.

In [3]:
# Collecting the images for the B, V and R bands into 3 different variables.

imagesV = ccdproc.ImageFileCollection(".",glob_include='proc_NGC 2323_V_*')
for fnV in imagesV.files_filtered(PICTTYPE = 1): #printing image file names
    print(fnV)
imagesB = ccdproc.ImageFileCollection(".",glob_include='proc_NGC 2323_B_*')
for fnB in imagesB.files_filtered(PICTTYPE = 1): #printing image file names
    print(fnB)
imagesR = ccdproc.ImageFileCollection(".",glob_include='proc_NGC 2323_R_*')
for fnR in imagesR.files_filtered(PICTTYPE = 1): #printing image file names
    print(fnR)

proc_NGC 2323_V_10.000secs_00002245.fit
proc_NGC 2323_V_10.000secs_00002246.fit
proc_NGC 2323_V_10.000secs_00002247.fit
proc_NGC 2323_V_10.000secs_00002248.fit
proc_NGC 2323_V_10.000secs_00002249.fit
proc_NGC 2323_V_10.000secs_00002250.fit
proc_NGC 2323_V_10.000secs_00002251.fit
proc_NGC 2323_V_10.000secs_00002252.fit
proc_NGC 2323_V_10.000secs_00002253.fit
proc_NGC 2323_V_10.000secs_00002254.fit
proc_NGC 2323_V_120.000secs_00002270.fit
proc_NGC 2323_V_120.000secs_00002271.fit
proc_NGC 2323_V_120.000secs_00002272.fit
proc_NGC 2323_V_120.000secs_00002273.fit
proc_NGC 2323_V_120.000secs_00002274.fit
proc_NGC 2323_B_10.000secs_00002235.fit
proc_NGC 2323_B_10.000secs_00002236.fit
proc_NGC 2323_B_10.000secs_00002237.fit
proc_NGC 2323_B_10.000secs_00002238.fit
proc_NGC 2323_B_10.000secs_00002239.fit
proc_NGC 2323_B_10.000secs_00002240.fit
proc_NGC 2323_B_10.000secs_00002241.fit
proc_NGC 2323_B_10.000secs_00002242.fit
proc_NGC 2323_B_10.000secs_00002243.fit
proc_NGC 2323_B_10.000secs_00002244

In [4]:
# Reading the data for the 3 different bands in adu

scimV = [CCDData.read(fnV, unit = "adu") for fnV in imagesV.files_filtered(PICTTYPE = 1)] #Loading image data
scimB = [CCDData.read(fnB, unit = "adu") for fnB in imagesB.files_filtered(PICTTYPE = 1)] #Loading image data
scimR = [CCDData.read(fnR, unit = "adu") for fnR in imagesR.files_filtered(PICTTYPE = 1)] #Loading image data

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


# Shifting Images

The centroid function takes the first image from the given bands list and returns the x, y location of it, taking into account the pixel size of the image.

In [20]:
# Function to find the centoind to shift the images to

def centroid(scim: list) -> int: #renamed files
    temp=scim[0].copy()
    temp=temp-np.ma.median(scim)
    
    x1, y1 = centroid_2dg(temp[150:225,775:850])
    return x1+775, y1+150

In [23]:
# finding the centroids

v = centroid(scimV)
b = centroid(scimB)
r = centroid(scimR)
print("V-Band Centroid Location: {}".format(v))
print("B-Band Centroid Location: {}".format(b))
print("R-Band Centroid Location: {}".format(r))


V-Band Centroid Location: (791.0473116976607, 159.01985840126824)
B-Band Centroid Location: (798.7425558271386, 158.2878981994606)
R-Band Centroid Location: (797.2534730445241, 158.46304458956482)


The shifting function takes an image collection and its respective list of image data and performs the shifting on them. The x and y offest was determined by running the above function and using trial and error to find the best offset to perform the shift with. The fuction then runs through each image of the scim list and finally saves creates new files with the notation sproc to indicate the images have already been shifted

In [8]:
# FUnction to shift the images using the found centroind

def shifting(images: ccdproc.ImageFileCollection, scim: list) -> None: #files are image collection, used to rename and save files
    
    
    xoffset = 760  # x edge of the box 
    yoffset = 140 # y edge of the box
    xbox = 100       # x length of the box
    ybox = 100   
    shiftx = []
    shifty = []
    
    shiftdata = {
        'Centroid (x)': [], 
        'Centroid (y)': [], 
        'Shift (x)': [], 
        'Shift (y)': []
    }
    
    
    for idx, thisimage in enumerate(scim): 
        temp = thisimage.copy()
        temp = temp-np.ma.median(temp)
        x1, y1 = centroid_2dg( temp[yoffset : yoffset + ybox, xoffset : xoffset + xbox]) #forming the centroid
        #print(x1 +xoffset, y1 + yoffset )
        shiftx.append(x1 + xoffset) #fills the shift list
        shifty.append(y1 + yoffset) #fills the shift list
        
    
    
    shifts=[] #creates new lst
    for i in range(len(shiftx)):
        shifts.append((np.rint(shiftx[0]-shiftx[i]),np.rint(shifty[0]-shifty[i]))) #subtracts each images centroid location from
        #the each image of NGC2362 and appends it to the list shift as a touple
    print(shifts)
    xoffset = 760  # x edge of the box 
    yoffset = 140 # y edge of the box
    xbox = 100       # x length of the box
    ybox = 100       # y length of the box
    

    # For the images in scim, it applies the shift on the centroid
    for idx, thisimage in enumerate(scim): 
        yxshifts=(shifts[idx][1], shifts[idx][0]) # Note the y-x convention being used here and in the following command.
        temp = shift(scim[idx], yxshifts, order=0, mode='constant', cval=-1000)
        temp=temp-np.ma.median(temp) #calculates median of temp

        # prints the new location of the images centroids and prints how much they have been shifted on each axis
        x1, y1 = centroid_2dg(temp[yoffset:yoffset+ybox,xoffset:xoffset+xbox])
        shiftdata['Centroid (x)'].append(x1+xoffset)
        shiftdata['Centroid (y)'].append(y1+yoffset)
        shiftdata['Shift (x)'].append(shifts[idx][0]), 
        shiftdata['Shift (y)'].append(shifts[idx][1]) 
                
     
    newname=[] #creates a new list
    for fn in images.files_filtered(PICTTYPE=1):#for filename with image type 1
        newname.extend(["s"+fn])# add s to the start of the file so it goes from fit to fits
    print(newname)#prints the list of new names

    df = pd.DataFrame(data=shiftdata, index=newname)
    print(df)
    
    for idx, thisimage in enumerate(scim): 
        yxshifts=(shifts[idx][1], shifts[idx][0])
        temp = CCDData(shift(scim[idx], yxshifts, order=0, mode='constant', cval=-1000)-np.ma.median(scim[idx]), unit="adu")
        temp.header = scim[idx].header
        temp.write(newname[idx])
    
    

In [9]:
# Shifting the images

shifting(imagesV, scimV)
shifting(imagesB, scimB)
shifting(imagesR, scimR)

[(0.0, 0.0), (-0.0, -0.0), (2.0, -1.0), (3.0, -0.0), (4.0, -1.0), (6.0, -1.0), (4.0, -1.0), (4.0, -2.0), (3.0, -2.0), (1.0, -1.0), (-1.0, -7.0), (-1.0, -8.0), (-1.0, -8.0), (-1.0, -8.0), (-1.0, -8.0)]


['sproc_NGC 2323_V_10.000secs_00002245.fit', 'sproc_NGC 2323_V_10.000secs_00002246.fit', 'sproc_NGC 2323_V_10.000secs_00002247.fit', 'sproc_NGC 2323_V_10.000secs_00002248.fit', 'sproc_NGC 2323_V_10.000secs_00002249.fit', 'sproc_NGC 2323_V_10.000secs_00002250.fit', 'sproc_NGC 2323_V_10.000secs_00002251.fit', 'sproc_NGC 2323_V_10.000secs_00002252.fit', 'sproc_NGC 2323_V_10.000secs_00002253.fit', 'sproc_NGC 2323_V_10.000secs_00002254.fit', 'sproc_NGC 2323_V_120.000secs_00002270.fit', 'sproc_NGC 2323_V_120.000secs_00002271.fit', 'sproc_NGC 2323_V_120.000secs_00002272.fit', 'sproc_NGC 2323_V_120.000secs_00002273.fit', 'sproc_NGC 2323_V_120.000secs_00002274.fit']
                                           Centroid (x)  Centroid (y)  \
sproc_NGC 2323_V_10.000secs_00002245.fit     791.047599    159.019485   
sproc_NGC 2323_V_10.000secs_00002246.fit     791.199405    159.355005   
sproc_NGC 2323_V_10.000secs_00002247.fit     791.137373    158.550174   
sproc_NGC 2323_V_10.000secs_00002248.fit  

OSError: File 'sproc_NGC 2323_V_10.000secs_00002245.fit' already exists.

# Scaling

The collectsproc function just creates a collection of the sproc images with the given band and creates a list of the given images. It then prints the first image so it can be check manually.

In [24]:
def collectsproc(band: str) -> ccdproc.ImageFileCollection:
    images = ccdproc.ImageFileCollection(".",glob_include = 'sproc_NGC 2323_'+band+'*') 
    for fn in images.files_filtered(PICTTYPE = 1):
        print(fn)
    scim = [CCDData.read(fn, unit = "adu") for fn in images.files_filtered(PICTTYPE = 1)]
    print(scim[0])
    return scim

The aperturephotometry function takes in a band (V, B or R) and performs aperture photometry on the images with the respective band. Using the photometry it scales the images using an average and median fit. The function then combines the images together and saves two new files containing an average and median combined images

In [15]:
def apeturephotometry(band: str) -> None:
    
    scim = collectsproc(band)
    
    positions = [(794,167),(1526,1148), (83,947)]   #x-y notation
    apertures = CircularAperture(positions, r=20.0)

    phot_table = aperture_photometry(scim[0], apertures)
    print('\n')
    print(phot_table)
    
    phot_table=[] #create new table
    for idx, thisimage in enumerate(scim): # enumerates each image in scim
        phot_table.extend([aperture_photometry(thisimage, apertures)]) #performs appeture photometry on each image for each given star location
        print(idx, phot_table[idx]) #prints the stars location and the adu of the star
    
    
#     for idx, thisimage in enumerate(scim): 
#         print(idx) #prints the index
#         print(phot_table[6]['aperture_sum']/phot_table[idx]['aperture_sum'])#prints the appeturesum/photo_tables values of image 7
#         print(np.ma.median(phot_table[6]['aperture_sum']/phot_table[idx]['aperture_sum']))#prints the invalid data

    images = ccdproc.ImageFileCollection(".",glob_include = 'sproc_NGC 2323_'+band+'*')
    scim = [CCDData.read(fn) for fn in images.files_filtered()]
    for idx, thisimage in enumerate(scim): 
        m = np.ma.median(phot_table[6]['aperture_sum'] / phot_table[idx]['aperture_sum']) 
        print(m) 
        scim[idx] = scim[idx].multiply(m * u.adu) 

    sci_average = ccdproc.combine(scim, method = 'average',dtype = np.float32, #combines all the images using an average method
                                  minmax_clip = True, minmax_clip_min = -500)
    sci_average.write("NGC_2323_"+band+"_average.fits") #wirtes the average image into a fits file

    sci_median = ccdproc.combine(scim, method = 'median',dtype = np.float32,  #combines all images usinging a median method
                                 minmax_clip = True, minmax_clip_min = -500)
    sci_median.write("NGC_2323_"+band+"_median.fits") #writes the median into a fits file

    del(scim)
    collected = gc.collect() #clears up memory space
    print('Check garbage collection', collected)

In [16]:
# Creating the scaled images

apeturephotometry('V')
apeturephotometry('B')
apeturephotometry('R')

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'.


sproc_NGC 2323_V_10.000secs_00002245.fit
sproc_NGC 2323_V_10.000secs_00002246.fit
sproc_NGC 2323_V_10.000secs_00002247.fit
sproc_NGC 2323_V_10.000secs_00002248.fit
sproc_NGC 2323_V_10.000secs_00002249.fit
sproc_NGC 2323_V_10.000secs_00002250.fit
sproc_NGC 2323_V_10.000secs_00002251.fit
sproc_NGC 2323_V_10.000secs_00002252.fit
sproc_NGC 2323_V_10.000secs_00002253.fit
sproc_NGC 2323_V_10.000secs_00002254.fit
sproc_NGC 2323_V_120.000secs_00002270.fit
sproc_NGC 2323_V_120.000secs_00002271.fit
sproc_NGC 2323_V_120.000secs_00002272.fit
sproc_NGC 2323_V_120.000secs_00002273.fit
sproc_NGC 2323_V_120.000secs_00002274.fit
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
[[ 28.351334   87.81384    61.067596  ...  12.912567  -33.605865
  -45.949295 ]
 [ 52.06482   -28.47467    76.25806   ... -17.903625   -7.1490326
   38.222595 ]
 [ 40.87787    24.965118  -12.1132965 ...  12.395294   30.2305
    3.5681   ]
 ...
 [-30.876862   69.41641   -34.294373  ... -14.236588  -48.16783
   -6.710861 ]
 [-63.471848  103.339294  -26.714645  ... -30.556808   49.10553
  -60.307556 ]
 [-20.72

Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'.


1.0071437232796632
0.9849160022545497
0.9982022039361224
0.9947939296754278
1.0020809423283286
0.9852603268657204
1.0
0.9994900538480708
0.9877207522218356
0.9904219581822703
0.2762846847599991
0.1873716452878317
0.19068406184290526
0.6110520336864708
0.610350286060772
0.7760809978349841
0.9463321163544206
2.5951262110326776
0.17032986055011126
0.1768780151533924
0.6226825126225697
Check garbage collection 192
